# Code Playground for Percept Data



<b> Data is required to converted into the BIDS-standard. </b>




### 0. Loading packages and functions, defining paths



In [1]:
# Importing Python and external packages
import os
import sys
import importlib
import json
from dataclasses import dataclass, field, fields
from itertools import compress
import csv
import pandas as pd
import numpy as np

import sklearn as sk
import scipy
import matplotlib.pyplot as plt
from scipy import signal

#mne
import mne_bids
import mne

In [2]:
# check some package versions for documentation and reproducability
print('Python sys', sys.version)
print('pandas', pd.__version__)
print('numpy', np.__version__)
print('mne_bids', mne_bids.__version__)
print('mne', mne.__version__)
print('sci-py', scipy.__version__)
print('sci-kit learn', sk.__version__)

Python sys 3.10.6 | packaged by conda-forge | (main, Aug 22 2022, 20:43:44) [Clang 13.0.1 ]
pandas 1.5.0
numpy 1.21.6
mne_bids 0.11
mne 1.2.0
sci-py 1.9.1
sci-kit learn 1.1.2


In [7]:
filepath = '/Users/jenniferbehnke/Coding/Perceive_output/ieeg_ses-2021080602000195_3MFU/sub021_ses-2021080602000195_run-BrainSense20210806020700.mat'
# define local storage directories

# check working directory
print(os.getcwd())

# get folder above a certain folder
print(os.path.dirname(os.getcwd()))

# change working directory to project-code folder
os.chdir('/Users/jenniferbehnke/Coding/Perceive_output/ieeg_ses-2021080602000195_3MFU/')


/Users/jenniferbehnke/Coding/PyPerceive/code/notebooks
/Users/jenniferbehnke/Coding/PyPerceive/code


## Pre-Work - Getting into the Files

In [23]:
# import file as MNE object
# Varvaras notebook: raw = mne.io.read_raw_fieldtrip('sub021_ses-2021080602000195_run-BrainSense20210806020700.mat',info = None)

data = mne.io.read_raw_fieldtrip(
    filepath,
    info={},
    data_name='data',
)
data.info

Creating RawArray with float64 data, n_channels=6, n_times=3688
    Range : 0 ... 3687 =      0.000 ...    14.748 secs
Ready.


/var/folders/d4/z737wy2x2y309dsbbzk9y8yr0000gn/T/ipykernel_4884/1624976735.py:4: RuntimeWarning: The supplied FieldTrip structure does not have an elec or grad field. No channel locations will extracted and the kind of channel might be inaccurate.
  data = mne.io.read_raw_fieldtrip(
/var/folders/d4/z737wy2x2y309dsbbzk9y8yr0000gn/T/ipykernel_4884/1624976735.py:4: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_13. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(
/var/folders/d4/z737wy2x2y309dsbbzk9y8yr0000gn/T/ipykernel_4884/1624976735.py:4: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_R_02. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(
/var/folders/d4/z737wy2x2y309dsbbzk9y8yr0000gn/T/ipykernel_4884/1624976735.py:4: RuntimeWarning: Cannot guess the correct type of channel LFP_Stn_L_13_PEAK15Hz_THR20-30_AVG3000ms. Making it a MISC channel.
  data = mne.io.read_raw_fieldtrip(
/var/folders/d4/z737wy2x2y309dsbbzk9y8yr0000gn

<Info | 8 non-empty values
 bads: []
 ch_names: LFP_Stn_L_13, LFP_Stn_R_02, ...
 chs: 4 misc, 2 Stimulus
 custom_ref_applied: False
 dig: 0 items
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 6
 projs: []
 sfreq: 250.0 Hz
>

In [10]:
# load raw-mat file into python
dat = scipy.io.loadmat(filepath)

In [11]:
print(raw) #here you can see the number of samples and the time
print(raw.info) #ch_names, sfreq, nchan.
n_time_samps = raw.n_times #nsamples
time_secs = raw.times #timepoints
raw.info.keys()
ch_names = raw.ch_names
n_chan = len(ch_names) 

<RawArray | 6 x 3688 (14.7 s), ~184 kB, data loaded>
<Info | 8 non-empty values
 bads: []
 ch_names: LFP_Stn_L_13, LFP_Stn_R_02, ...
 chs: 4 misc, 2 Stimulus
 custom_ref_applied: False
 dig: 0 items
 highpass: 0.0 Hz
 lowpass: 125.0 Hz
 meas_date: unspecified
 nchan: 6
 projs: []
 sfreq: 250.0 Hz
>


In [17]:
print(ch_names)
#print(raw._data[2,:])

raw_data = raw.get_data()
print(raw_data.shape)

#mydat = raw.get_data(picks='LFP_R_13_STN')
#print(mydat)
#fig, ax = plt.subplots(figsize=[15, 5])
#ax.plot(mydat)
#plt.show()

['LFP_Stn_L_13', 'LFP_Stn_R_02', 'LFP_Stn_L_13_PEAK15Hz_THR20-30_AVG3000ms', 'LFP_Stn_R_02_PEAK15Hz_THR20-30_AVG3000ms', 'STIM_L_125Hz_60us', 'STIM_R_125Hz_60us']
(6, 3688)


In [20]:
raw.__dict__

{'_data': array([[ -4.44034504,  -3.64336003,  -4.44034504, ...,  10.9300801 ,
           9.67767509,  10.3608051 ],
        [ -5.80660505,  -6.26202506,  -3.30179503, ...,   7.85599507,
           8.31141508,   7.74214007],
        [482.        , 482.        , 482.        , ...,          nan,
                  nan,          nan],
        [587.        , 587.        , 587.        , ...,          nan,
                  nan,          nan],
        [  0.        ,   0.        ,   0.        , ...,          nan,
                  nan,          nan],
        [  0.        ,   0.        ,   0.        , ...,          nan,
                  nan,          nan]]),
 'preload': True,
 '_last_samps': array([3687]),
 '_first_samps': array([0]),
 'info': <Info | 8 non-empty values
  bads: []
  ch_names: LFP_Stn_L_13, LFP_Stn_R_02, ...
  chs: 4 misc, 2 Stimulus
  custom_ref_applied: False
  dig: 0 items
  highpass: 0.0 Hz
  lowpass: 125.0 Hz
  meas_date: unspecified
  nchan: 6
  projs: []
  sfreq: 250.0 H

In [21]:
# principe of dictionary
a = {
    'a': np.arange(10),
    'b': 'bbbbbbb',
    'c': {
        'd': np.arange(10),
    }
}

IndentationError: unexpected indent (4018942458.py, line 3)

In [44]:
data_as_dict = vars(data)

print(data_as_dict.keys())

# plt.plot(data._data[0, :])
# plt.plot(data._data[1, :])

dict_keys(['_data', 'preload', '_last_samps', '_first_samps', 'info', 'buffer_size_sec', '_cals', '_raw_extras', '_read_picks', '_read_comp_grade', '_comp', '_filenames', 'orig_format', '_orig_units', '_projectors', '_projector', '_dtype_', '_annotations', '_cropped_samp', '_init_kwargs'])


In [43]:
data._annotations

<Annotations | 0 segments>